In [15]:
import torch
from AllCode import SolverV2_opt, SceneBuilder
from TuRBO import turbo_optimize_density
import numpy as np
import pandas as pd

In [16]:
data = pd.read_pickle('RegridExpData.pickle')
Ndata = data['N']
Sdata = data['S']
# N0 = Ndata[0]
# S0 = Sdata[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64

# # your grid & data
rho  = torch.linspace(1e-3, 1.2, 151, device=device, dtype=dtype)
time = torch.linspace(1e-3, 0.065317, 161, device=device, dtype=dtype)


# #Spatial Source
# A_SR, Rc_SR, o_SR = (1, 1, 1)
# SR = A_SR*torch.exp(-((rho - Rc_SR)/o_SR)**2) #! 72.9s / 4090.7s

# #Temporal Source
# A_ST, f_ST, p_ST = (1, 1, 1)
# ST = A_ST*torch.sin(2*np.pi*f_ST*time + p_ST) + 1.0  # ensure non-negative  #! 59.2s / 4090.7s

# #Initial Density
# A_N0, Xs_N0, H_N0, a_N0, B_N0 = (1, 1, 1, 1, 1)
# N0Funca = lambda r, a: ((1 + a*r)*torch.exp(r) - torch.exp(-r))/(torch.exp(r) + torch.exp(-r))
# N0Funcb = lambda r, xs, H: (xs - r)/H
# N0Func = lambda r, a, xs, H, A, B: A*N0Funca(N0Funcb(r,xs,H),a) + B

# #Edge Boundary Condition
# A_sign, A_mag = (1, 1)
# if A_sign >= 0:
# 	A_sign = 1.0
# else:
# 	A_sign = -1.0
# A_edge = torch.tensor([A_sign*A_mag], device=device, dtype=dtype)

# __1 = torch.tensor(N0, device=device, dtype=dtype)  # (Nt,)
# __2 = torch.tensor(N0.T, device=device, dtype=dtype)  # (Nt,)
# S0 = torch.tensor(S0, device=device, dtype=dtype)  # (Nr,)

# N_target = __2.to(device, dtype)  # (Nr,Nt)
# print(N_target.shape)
# SR = S0[0, :].to(device, dtype)            # (Nr,)
# print(SR.shape)
# ST = S0[:, 125].to(device, dtype)          # (Nt,)
# print(ST.shape)
# N0 = __1[0, :]
# N0 = N0.to(device, dtype)                  # (Nr,)
# print(N0.shape)
# A_edge = torch.tensor([0.0], device=device, dtype=dtype)

In [18]:
...

Ellipsis

In [19]:
# --- CORRECTED DATA LOADING (Separating Magnitude and Modulation) ---

# (This assumes Ndata and Sdata are loaded from your 'other cell')
# Ndata shape is (N_shots, 161, 151)
# Sdata shape is (N_shots, 161, 151)

# 1. N_target (Correct)
N_target_data = torch.tensor(Ndata[0], device=device, dtype=dtype)
N_target = N_target_data.T.contiguous()

# 2. N0 (Correct)
N0 = N_target_data[0, :].contiguous()

# 3. A_edge (Correct)
A_sign, A_mag = (1, 1) 
if A_sign >= 0: A_sign = 1.0
else: A_sign = -1.0
A_edge = torch.tensor([A_sign * A_mag], device=device, dtype=dtype)

# 4. Sdata (Correct)
Sdata_shot = torch.tensor(Sdata[0], device=device, dtype=dtype) # (161, 151)

# --- 5. THE FIX (Your Idea) ---

# SR (Spatial Profile): Let this carry the full magnitude.
# We average over time (axis=0) to get the avg spatial profile.
SR = Sdata_shot.mean(axis=0) # Shape (151,), min/max ~1e22
# Clamp to avoid negative source, which is non-physical
SR.clamp_min_(0.0)

# ST (Temporal Modulation): This must be unitless (centered at 1.0).
# We average over space (axis=1) to get the temporal profile.
ST_raw = Sdata_shot.mean(axis=1) # Shape (161,), min/max ~1e22
ST_raw.clamp_min_(1e-30) # Avoid division by zero if source is ever zero

# Get the average value of the temporal profile
ST_mean = ST_raw.mean() # A single scalar, ~1e22

# Divide by the mean to create a unitless modulation
ST = ST_raw / ST_mean # Shape (161,), min/max ~1.0

# --- END OF FIX ---

# --- Run the Verification ---
print(f"--- VERIFYING INPUTS ---")
print(f"rho shape: {rho.shape} | time shape: {time.shape}")
print(f"N_target shape: {N_target.shape} (Expected: {rho.shape[0]}, {time.shape[0]})")
print(f"SR shape: {SR.shape} (Expected: {rho.shape[0]})")
print(f"ST shape: {ST.shape} (Expected: {time.shape[0]})")
print(f"N0 shape: {N0.shape} (Expected: {rho.shape[0]})")

print(f"\nN_target min/max/has_nan: {N_target.min():.2e}, {N_target.max():.2e}, {torch.isnan(N_target).any()}")
print(f"N0 min/max/has_nan: {N0.min():.2e}, {N0.max():.2e}, {torch.isnan(N0).any()}")
print(f"\n--- CHECK MAGNITUDES (This is the fix) ---")
print(f"SR min/max/has_nan: {SR.min():.2e}, {SR.max():.2e}, {torch.isnan(SR).any()}")
print(f"ST min/max/has_nan: {ST.min():.2e}, {ST.max():.2e}, {torch.isnan(ST).any()}")

assert N_target.shape == (rho.shape[0], time.shape[0]), "N_target shape is WRONG"
assert SR.shape == rho.shape, "SR shape is WRONG"
assert ST.shape == time.shape, "ST shape is WRONG"
assert N0.shape == rho.shape, "N0 shape is WRONG"
assert not torch.isnan(N_target).any(), "N_target contains NaNs"
assert not torch.isnan(SR).any(), "SR contains NaNs"
assert not torch.isnan(ST).any(), "ST contains NaNs"
assert not torch.isnan(N0).any(), "N0 contains NaNs"
print("--- VERIFICATION PASSED ---")

--- VERIFYING INPUTS ---
rho shape: torch.Size([151]) | time shape: torch.Size([161])
N_target shape: torch.Size([151, 161]) (Expected: 151, 161)
SR shape: torch.Size([151]) (Expected: 151)
ST shape: torch.Size([161]) (Expected: 161)
N0 shape: torch.Size([151]) (Expected: 151)

N_target min/max/has_nan: 2.98e+19, 2.98e+20, False
N0 min/max/has_nan: 2.98e+19, 2.50e+20, False

--- CHECK MAGNITUDES (This is the fix) ---
SR min/max/has_nan: 0.00e+00, 8.30e+22, False
ST min/max/has_nan: 8.46e-01, 1.15e+00, False
--- VERIFICATION PASSED ---


In [20]:
solver = SolverV2_opt()

res = turbo_optimize_density(
    rho=rho, time=time, N_target=N_target,
    solver=solver, SR=SR, ST=ST, N0=N0, A_edge=A_edge,
    n_knots=6, D_bounds=(1e-2, 0.1), V_bounds=(-0.1, 0.1),
    n_steps=50, batch_q=8, top_k_return=5,
)

print("Best losses:", [round(r['loss'],3) for r in res['best']])
D0, V0 = res['best'][0]['D'], res['best'][0]['V']
N0pred = res['best'][0]['N_pred']


Using device: cuda
[TuRBO] step 01: added 8 | TR_len=0.800 | best=7.417e+41
[TuRBO] step 02: added 8 | TR_len=0.800 | best=7.285e+41
[TuRBO] step 03: added 8 | TR_len=0.800 | best=7.285e+41
[TuRBO] step 04: added 8 | TR_len=0.800 | best=7.285e+41
[TuRBO] step 05: added 8 | TR_len=0.400 | best=7.285e+41
[TuRBO] step 06: added 8 | TR_len=0.400 | best=7.285e+41
[TuRBO] step 07: added 8 | TR_len=0.400 | best=7.263e+41
[TuRBO] step 08: added 8 | TR_len=0.400 | best=7.263e+41
[TuRBO] step 09: added 8 | TR_len=0.400 | best=7.263e+41
[TuRBO] step 10: added 8 | TR_len=0.200 | best=7.263e+41
[TuRBO] step 11: added 8 | TR_len=0.200 | best=7.263e+41
[TuRBO] step 12: added 8 | TR_len=0.200 | best=7.263e+41
[TuRBO] step 13: added 8 | TR_len=0.100 | best=7.263e+41
[TuRBO] step 14: added 8 | TR_len=0.100 | best=7.263e+41
[TuRBO] step 15: added 8 | TR_len=0.100 | best=7.263e+41
[TuRBO] step 16: added 8 | TR_len=0.050 | best=7.263e+41
[TuRBO] step 17: added 8 | TR_len=0.050 | best=7.263e+41
[TuRBO] step

KeyboardInterrupt: 